<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Istanbul</font></h1>

### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* Introduction: Business Problem
* Data
* Methodology
* Analysis
* Results and Discussion
* Conclusion

## Introduction: Business Problem <a name="introduction"></a>

Moonlight Tours is a travel agency for people to travel abroad for holidays with packages. They provide different packages i.e Classic, Premium and Luxury and their Luxury package includes Hotel stay, food at top restaurants and sightseeing of top venues around the city. They make customer satisfaction their top priority and are famous for these services. They are branching out to Turkey as their new travel destination. To maintain their quality they are to make list of nearest venues with restaurants,hotels, cafes,shopping stores for their customers.


## Data <a name="data"></a>

We will use data provided by the Foursquare API using their location services. We will explore the city of Istanbul, Turkey. We will further explore the neighborhoods in the city and group them according to the top venues which include restaurants, cafes, hotels and other public places to visit. 

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
!conda install -c anaconda beautifulsoup4 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         154 KB  anaconda

The following packages will be UPDATED:

    certifi: 2019.9.11-py36_0  conda-forge --> 2019.9.11-py36_0 anaconda
    openssl: 1.1.1c-h516909a_0 conda-forge --> 1.1.1-h7b6447c_0 anaconda


certifi-2019.9.11    | 154 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [15]:
!conda install lxml --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         154 KB

The following packages will be UPDATED:

    certifi: 2019.9.11-py36_0  conda-forge --> 2019.9.11-py36_0 
    openssl: 1.1.1c-h516909a_0 conda-forge --> 1.1.1d-h7b6447c_3


certifi-2019.9.11    | 154 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [3]:
!conda install -c conda-forge geopy --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge

The following packages will be UPDATED:

    certifi: 2019.9.11-py36_0 anaconda --> 2019.9.11-py36_0  conda-forge

The following packages will be DOWNGRADED:

    openssl: 1.1.1-h7b6447c_0 anaconda --> 1.1.1c-h516909a_0 conda-forge


certifi-2019.9.11    | 147 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [4]:
!conda install -c conda-forge folium=0.5.0 --yes 

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [36]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

from bs4 import BeautifulSoup

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


The next task is essentially transforming this data of nested Python dictionaries into a pandas dataframe. So let's start by creating an empty dataframe.

#### Use geopy library to get the latitude and longitude values of Istanbul.

In [37]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_districts_of_Istanbul")
soup = BeautifulSoup(page.content, 'html.parser')

 Scrap the data and Tranform the data into a *pandas* dataframe

In [38]:
table = soup.find('tbody')
rows = table.select('tr')
row = [r.get_text() for r in rows]
df = pd.DataFrame(row)
df1 = df[0].str.split('\n', expand=True)
df2 = df1.rename(columns=df1.iloc[0])
df3 = df2.drop(df2.index[0])
df3.head()

,,District,,Population (2018),,Area (km²),,Density (per km²),
1,,Adalar,,"16,119",,11.05,,"1,458",
2,,Arnavutköy,,"270,549",,450.35,,600,
3,,Ataşehir,,"416,318",,25.20,,"16,520",
4,,Avcılar,,"435,625",,42.01,,"10,369",
5,,Bağcılar,,"734,369",,22.36,,"32,842",


Chnage the name of the column

In [39]:
df_Neighbourhoods = df3[['District']]
df_Neighbourhoods.rename(columns={'District': 'Neighborhood'},inplace=True)

In [40]:
df_Neighbourhoods = pd.DataFrame(df_Neighbourhoods)
df_Neighbourhoods.head()

,Neighborhood
1,Adalar
2,Arnavutköy
3,Ataşehir
4,Avcılar
5,Bağcılar


The next task is to get the coordinates and assign to these neighbourhoods in a new dataframe. So let's start by creating an empty dataframe.

In [41]:
# define the dataframe columns
column_names = [ 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Take a look at the empty dataframe to confirm that the columns are as intended.

In [42]:
neighborhoods.head()

,Neighborhood,Latitude,Longitude


Then let's loop through the data and fill the dataframe one row at a time

In [43]:
from time import sleep

In [ ]:
for index, row in df_Neighbourhoods.iterrows():
    print(row['Neighborhood'])
    name = row['Neighborhood'] +',TR'
    geolocator = Nominatim(user_agent="ny_explorer")
    #location = geolocator.geocode(name)
    location = geolocator.geocode(name, timeout=10)
    latitude = location.latitude
    longitude = location.longitude
    
    neighborhood_name= name
    neighborhood_lat = latitude
    neighborhood_lon = longitude
    
    neighborhoods = neighborhoods.append({
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)


 Quickly examine the resulting dataframe.

In [45]:
neighborhoods.head()

,Neighborhood,Latitude,Longitude
0,"Adalar,TR",40.875931,29.094742
1,"Arnavutköy,TR",41.068394,29.041154
2,"Ataşehir,TR",40.984749,29.106720
3,"Avcılar,TR",40.980135,28.717547
4,"Bağcılar,TR",41.033899,28.857898


In [48]:
print('The dataframe has {} Neighborhood.'.format(
        len(neighborhoods['Neighborhood'].unique())
      
    )
)

The dataframe has 39 Neighborhood.


In [47]:
#### Use geopy library to get the latitude and longitude values of Istanbul.

In [49]:
address = 'Istanbul, TR'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
sleep(1)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Istanbul are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Istanbul are 41.0096334, 28.9651646.


#### Create a map of Istanbul with neighborhoods superimposed on top.

In [50]:
# create map of Istanbul using latitude and longitude values
map_Istanbul = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng,  neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'],  neighborhoods['Neighborhood']):
    label = neighborhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Istanbul)  
    
map_Istanbul

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below.

**Folium** is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Manhattan. So let's slice the original dataframe and create a new dataframe of the Manhattan data.

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [51]:
CLIENT_ID = 'I3SN5XEQSPO4SV0FBRTDLXTWNVVL55RKXAWF2FPAKBJ3XVO5' # your Foursquare ID
CLIENT_SECRET = 'ZNYIKWFVCQBHBFJSJ11XYIQEDP3NY1IRUDJYWC1KYXIDNJC4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: I3SN5XEQSPO4SV0FBRTDLXTWNVVL55RKXAWF2FPAKBJ3XVO5
CLIENT_SECRET:ZNYIKWFVCQBHBFJSJ11XYIQEDP3NY1IRUDJYWC1KYXIDNJC4


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [52]:
neighborhoods.loc[0, 'Neighborhood']

'Adalar,TR'

Get the neighborhood's latitude and longitude values.

In [53]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Adalar,TR are 40.8759308, 29.0947421.


#### Now, let's get the top 100 venues that are in Adalar within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**.

In [54]:
# type your answer here


LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=I3SN5XEQSPO4SV0FBRTDLXTWNVVL55RKXAWF2FPAKBJ3XVO5&client_secret=ZNYIKWFVCQBHBFJSJ11XYIQEDP3NY1IRUDJYWC1KYXIDNJC4&v=20180605&ll=40.8759308,29.0947421&radius=500&limit=100'

Send the GET request and examine the resutls

In [55]:
results = requests.get(url).json()
#results

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [56]:

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [57]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
#nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,L'isola Guesthouse,Bed & Breakfast,40.877038,29.096136
1,İnönü Evi Müzesi,History Museum,40.878251,29.093647
2,Luz Café,Café,40.877528,29.097877
3,Heybeliada Şafak Askeri Gazino,Restaurant,40.873609,29.099478
4,Merit Halki Palace Hotel,Hotel,40.878802,29.090974


And how many venues were returned by Foursquare?

In [58]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

25 venues were returned by Foursquare.


#### Explore Neighborhoods in Istanbul

##### Let's create a function to repeat the same process to all the neighborhoods in Istanbul

In [59]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *Istanbul_venues*.

In [60]:

Istanbul_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )


Adalar,TR
Arnavutköy,TR
Ataşehir,TR
Avcılar,TR
Bağcılar,TR
Bahçelievler,TR
Bakırköy,TR
Başakşehir,TR
Bayrampaşa,TR
Beşiktaş,TR
Beykoz,TR
Beylikdüzü,TR
Beyoğlu,TR
Büyükçekmece,TR
Çatalca,TR
Çekmeköy,TR
Esenler,TR
Esenyurt,TR
Eyüp,TR
Fatih,TR
Gaziosmanpaşa,TR
Güngören,TR
Kadıköy,TR
Kağıthane,TR
Kartal,TR
Küçükçekmece,TR
Maltepe,TR
Pendik,TR
Sancaktepe,TR
Sarıyer,TR
Silivri,TR
Sultanbeyli,TR
Sultangazi,TR
Şile,TR
Şişli,TR
Tuzla,TR
Ümraniye,TR
Üsküdar,TR
Zeytinburnu,TR


check the size of dataframe

In [76]:
print(Istanbul_venues.shape)
Istanbul_venues.head()

(2233, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Adalar,TR",40.875931,29.094742,L'isola Guesthouse,40.877038,29.096136,Bed & Breakfast
1,"Adalar,TR",40.875931,29.094742,İnönü Evi Müzesi,40.878251,29.093647,History Museum
2,"Adalar,TR",40.875931,29.094742,Luz Café,40.877528,29.097877,Café
3,"Adalar,TR",40.875931,29.094742,Heybeliada Şafak Askeri Gazino,40.873609,29.099478,Restaurant
4,"Adalar,TR",40.875931,29.094742,Merit Halki Palace Hotel,40.878802,29.090974,Hotel


In [88]:
Istanbul_venues2 = Istanbul_venues.head(1000)
Istanbul_venues2.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Adalar,TR",40.875931,29.094742,L'isola Guesthouse,40.877038,29.096136,Bed & Breakfast
1,"Adalar,TR",40.875931,29.094742,İnönü Evi Müzesi,40.878251,29.093647,History Museum
2,"Adalar,TR",40.875931,29.094742,Luz Café,40.877528,29.097877,Café
3,"Adalar,TR",40.875931,29.094742,Heybeliada Şafak Askeri Gazino,40.873609,29.099478,Restaurant
4,"Adalar,TR",40.875931,29.094742,Merit Halki Palace Hotel,40.878802,29.090974,Hotel


#### Show on Map top 200 Venues in Istanbul

In [89]:
# create map of Istanbul using latitude and longitude values
map_IstanbulVenues = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=10)

# add markers to map
for lat, lng,  neighborhood, cat in zip(Istanbul_venues2['Venue Latitude'], Istanbul_venues2['Venue Longitude'],  Istanbul_venues2['Venue'],Istanbul_venues2['Venue Category']):
    label = neighborhood + ','+ cat
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_IstanbulVenues)  
    
map_IstanbulVenues

Let's check how many venues were returned for each neighborhood

In [94]:
Istanbul_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adalar,TR",25,25,25,25,25,25
"Arnavutköy,TR",79,79,79,79,79,79
"Ataşehir,TR",37,37,37,37,37,37
"Avcılar,TR",100,100,100,100,100,100
"Bahçelievler,TR",85,85,85,85,85,85
"Bakırköy,TR",41,41,41,41,41,41
"Bayrampaşa,TR",43,43,43,43,43,43
"Bağcılar,TR",89,89,89,89,89,89
"Başakşehir,TR",20,20,20,20,20,20


Let's find out how many unique categories can be curated from all the returned venues

In [95]:
print('There are {} uniques categories.'.format(len(Istanbul_venues['Venue Category'].unique())))

There are 251 uniques categories.


##  Analyze Each Neighborhood

In [97]:
# one hot encoding
Istanbul_onehot = pd.get_dummies(Istanbul_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Istanbul_onehot['Neighborhood'] = Istanbul_onehot['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Istanbul_onehot.columns[-1]] + list(Istanbul_onehot.columns[:-1])
Istanbul_onehot = Istanbul_onehot[fixed_columns]

Istanbul_onehot.head()

,Women's Store,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Bathing Area,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Borek Place,Botanical Garden,Boutique,Breakfast Spot,Brewery,Bridal Shop,Buffet,Building,Burger Joint,Burrito Place,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Candy Store,Caribbean Restaurant,Casino,Cave,Cheese Shop,Chinese Restaurant,Chocolate Shop,Cigkofte Place,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Classroom,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Conference,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Entertainment Service,Fabric Shop,Factory,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Fish Taverna,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,Gift Shop,Gourmet Shop,Gozleme Place,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indie Movie Theater,Insurance Office,Internet Cafe,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kofte Place,Kokoreç Restaurant,Lake,Leather Goods Store,Lighthouse,Lounge,Manti Place,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Meyhane,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mosque,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,Nightclub,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Event Space,Paintball Field,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pide Place,Pie Shop,Pilavcı,Pizza Place,Plaza,Pool,Pool Hall,Preschool,Print Shop,Pub,Public Art,Radio Station,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Rest Area,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Souvenir Shop,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Taco Place,Tailor Shop,Tattoo Parlor,Taxi Stand,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Tiki Bar,Trail,Tram Station,Tree,Tunnel,Turkish Home Cooking Restaurant,Turkish Restaurant,Turşucu,Vegetarian / Vegan Restaurant,Video Game Store,Volcano,Volleyball Court,Warehouse Store,Water Park,Waterfront,Wedding Hall,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

check the size

In [98]:
Istanbul_onehot.shape

(2233, 251)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [99]:
Istanbul_grouped = Istanbul_onehot.groupby('Neighborhood').mean().reset_index()
Istanbul_grouped

,Neighborhood,Women's Store,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Bathing Area,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Borek Place,Botanical Garden,Boutique,Breakfast Spot,Brewery,Bridal Shop,Buffet,Building,Burger Joint,Burrito Place,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Candy Store,Caribbean Restaurant,Casino,Cave,Cheese Shop,Chinese Restaurant,Chocolate Shop,Cigkofte Place,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Classroom,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Conference,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Entertainment Service,Fabric Shop,Factory,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Fish Taverna,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,Gift Shop,Gourmet Shop,Gozleme Place,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indie Movie Theater,Insurance Office,Internet Cafe,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kofte Place,Kokoreç Restaurant,Lake,Leather Goods Store,Lighthouse,Lounge,Manti Place,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Meyhane,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mosque,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Nightclub,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Event Space,Paintball Field,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pide Place,Pie Shop,Pilavcı,Pizza Place,Plaza,Pool,Pool Hall,Preschool,Print Shop,Pub,Public Art,Radio Station,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Rest Area,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Souvenir Shop,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Taco Place,Tailor Shop,Tattoo Parlor,Taxi Stand,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Tiki Bar,Trail,Tram Station,Tree,Tunnel,Turkish Home Cooking Restaurant,Turkish Restaurant,Turşucu,Vegetarian / Vegan Restaurant,Video Game Store,Volcano,Volleyball Court,Warehouse Store,Water Park,Waterfront,Wedding Hall,Wine Bar,Wings Joint
0,0,0.004032,0.000448,0.000896,0.000448,0.000448,0.000448,0.000896,0.000448,0.009857,0.000448,0.007168,0.000896,0.001344,0.000896,0.001344,0.000448,0.004032,0.000896,0.000448,0.00224,0.000896,0.001792,0.022401,0.014337,0.001344,0.001344,0.000448,0.004032,0.001792,0.000448,0.00448,0.001344,0.000896,0.00672,0.001792,0.007168,0.001344,0.000896,0.005376,0.011201,0.000896,0.001344,0.003584,0.000896,0.008513,0.000448,0.000448,0.000896,0.155466,0.000448,0.000448,0.001792,0.000448,0.000896,0.000448,0.000448,0.001344,0.001344,0.003584,0.008961,0.000448,0.002688,0.02733,0.000448,0.000448,0.0035

In [100]:
Istanbul_grouped.shape

(2, 251)

In [101]:
num_top_venues = 5

for hood in Istanbul_grouped['Neighborhood']:
    print(hood)
    temp = Istanbul_grouped[Istanbul_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

0
                venue  freq
0                Café  0.16
1  Turkish Restaurant  0.05
2          Restaurant  0.04
3        Dessert Shop  0.03
4         Coffee Shop  0.03


1
           venue  freq
0  Women's Store   0.0
1    Pastry Shop   0.0
2    Music Venue   0.0
3     Nail Salon   0.0
4      Nightclub   0.0




In [102]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [105]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Istanbul_grouped['Neighborhood']

for ind in np.arange(Istanbul_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Istanbul_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Café,Turkish Restaurant,Restaurant,Seafood Restaurant,Dessert Shop,Coffee Shop,Steakhouse,Gym,Bakery,Gym / Fitness Center
1,1,Wings Joint,Dive Bar,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Factory,Fabric Shop,Entertainment Service


In [111]:
Istanbul_grouped.head()

,Women's Store,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Bathing Area,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Borek Place,Botanical Garden,Boutique,Breakfast Spot,Brewery,Bridal Shop,Buffet,Building,Burger Joint,Burrito Place,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Candy Store,Caribbean Restaurant,Casino,Cave,Cheese Shop,Chinese Restaurant,Chocolate Shop,Cigkofte Place,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Classroom,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Conference,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Doner Restaurant,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,English Restaurant,Entertainment Service,Fabric Shop,Factory,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Fish Taverna,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,General Entertainment,Gift Shop,Gourmet Shop,Gozleme Place,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indie Movie Theater,Insurance Office,Internet Cafe,Irish Pub,Italian Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Kofte Place,Kokoreç Restaurant,Lake,Leather Goods Store,Lighthouse,Lounge,Manti Place,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Meyhane,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mosque,Motorcycle Shop,Mountain,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Nightclub,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Event Space,Paintball Field,Paper / Office Supplies Store,Park,Parking,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Piano Bar,Pide Place,Pie Shop,Pilavcı,Pizza Place,Plaza,Pool,Pool Hall,Preschool,Print Shop,Pub,Public Art,Radio Station,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Rest Area,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Souvenir Shop,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Stationery Store,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Taco Place,Tailor Shop,Tattoo Parlor,Taxi Stand,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Tiki Bar,Trail,Tram Station,Tree,Tunnel,Turkish Home Cooking Restaurant,Turkish Restaurant,Turşucu,Vegetarian / Vegan Restaurant,Video Game Store,Volcano,Volleyball Court,Warehouse Store,Water Park,Waterfront,Wedding Hall,Wine Bar,Wings Joint
0,0.004032,0.000448,0.000896,0.000448,0.000448,0.000448,0.000896,0.000448,0.009857,0.000448,0.007168,0.000896,0.001344,0.000896,0.001344,0.000448,0.004032,0.000896,0.000448,0.00224,0.000896,0.001792,0.022401,0.014337,0.001344,0.001344,0.000448,0.004032,0.001792,0.000448,0.00448,0.001344,0.000896,0.00672,0.001792,0.007168,0.001344,0.000896,0.005376,0.011201,0.000896,0.001344,0.003584,0.000896,0.008513,0.000448,0.000448,0.000896,0.155466,0.000448,0.000448,0.001792,0.000448,0.000896,0.000448,0.000448,0.001344,0.001344,0.003584,0.008961,0.000448,0.002688,0.02733,0.000448,0.000448,0.003584,0.000896,0.0

In [115]:
# set number of clusters
kclusters = 2

#Istanbul_grouped_clustering = Istanbul_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Istanbul_grouped)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1], dtype=int32)